# Predict velocity values using deep learning

This notebook trains and uses an LSTM to predict velocity values. Modified from the code in [Encord's](https://encord.com/blog/time-series-predictions-with-recurrent-neural-networks/) post. An LSTM uses a memory cell to store information that the model uses to predict future values. Input, forget, and output gates control what is stored in, removed, and output from the LSTM. The LSTM's memory makes it ideal for predicting time series values; the lookback must be modified to encapsulate enough days to predict but not too many for overfitting.

---

### 1. Load packages and open dataset

Choose a point (`upstream`, `middle`, `terminus`).

In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [2]:
point_name = 'middle' # 'upstream', 'middle', or 'terminus'
input_file='../data/ai_ready/' + point_name + '_timeseries.csv'

# load the dataset, drop the series column
df = pd.read_csv(input_file)

df.drop(columns="series", inplace=True)
dataset=df['velocity'].values.reshape(-1, 1)
dataset.shape

(364, 1)

### 2. Preprocessing

Here, we convert the values into a matrix, normalize, and split into train and test data. Normalization ensures that predictions are scaled properly so that one data point does not dominate training or predictions. We split into 80/20 train/test data and use a look back of 15, meaning that the model will consider the previous 15 values to predict the next value.

In [3]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	print(len(dataX), len(dataY))
	return np.array(dataX), np.array(dataY)

In [4]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset.shape

(364, 1)

In [5]:
# split into train and test sets, 70% test data, 30% training data
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train.shape, test.shape

((254, 1), (110, 1))

In [6]:
# reshape into X=t and Y=t+1, timestep 5
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = trainX[:-1]
trainY = trainY[:-1]
trainX.shape, trainY.shape, testX.shape, testY.shape

248 248
104 104


((247, 5), (247,), (104, 5), (104,))

### 3. Train and test

Create the model and train for 5000 epochs. Save it to the `models` folder.

In [7]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

trainX.shape, trainY.shape, testX.shape, testY.shape

((247, 5, 1), (247,), (104, 5, 1), (104,))

In [8]:
# create and fit the LSTM network, optimizer=adam, 50 neurons, 2 LSTM layers
lstm_model = Sequential()
lstm_model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mse', optimizer='adam')
print(lstm_model.summary())
lstm_model.fit(trainX, trainY, epochs=500, batch_size=32, validation_data=(testX, testY), verbose=0)

/Users/cjense/miniconda3/envs/mlgeo-jensencc/lib/python3.10/site-packages/keras/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 100)         │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 100)         │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 100)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,451 (591.61 KB)

 Trainable params: 151,451 (591.61 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
# make predictions
trainPredict_lstm = lstm_model.predict(trainX)
testPredict_lstm = lstm_model.predict(testX)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [10]:
lstm_model.save('../models/' + point_name + '_lstm_model.keras')

In [11]:
# invert predictions
trainPredict_lstm = scaler.inverse_transform(trainPredict_lstm)
trainY = scaler.inverse_transform([trainY])
testPredict_lstm = scaler.inverse_transform(testPredict_lstm)
testY = scaler.inverse_transform([testY])

### 4. Analyze and plot predictions

Calculate RMSE for train and test, shift for plotting, and plot using Plotly. Note that the first 15 timesteps of both train and test are not accounted for in the prediction data since they were used to train the model.

In [12]:
# calculate root mean squared error
trainScore_lstm = math.sqrt(mean_squared_error(trainY[0], trainPredict_lstm[:,0]))
print('Train Score: %.2f RMSE' % (trainScore_lstm))
testScore_lstm = math.sqrt(mean_squared_error(testY[0], testPredict_lstm[:,0]))
print('Test Score: %.2f RMSE' % (testScore_lstm))

Train Score: 37.46 RMSE
Test Score: 42.18 RMSE


In [13]:
# shift train predictions for plotting
trainPredictPlot_lstm = np.empty_like(dataset)
trainPredictPlot_lstm[:, :] = np.nan
trainPredictPlot_lstm[look_back:len(trainPredict_lstm) + look_back, :] = trainPredict_lstm

In [14]:
# shift test predictions for plotting
testPredictPlot_lstm = np.empty_like(dataset)
testPredictPlot_lstm[:, :] = np.nan
testPredictPlot_lstm[len(trainPredict_lstm)+(look_back*2)+1:len(dataset)-2, :] = testPredict_lstm

In [15]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig.add_trace(go.Scatter(y=trainPredictPlot_lstm.flatten(), mode='lines', name='Train Predictions'))
fig.add_trace(go.Scatter(y=testPredictPlot_lstm.flatten(), mode='lines', name='Test Predictions'))

# Update layout for dark theme
fig.update_layout(title='LSTM Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
years = pd.to_datetime(df['time']).dt.year
unique_years = sorted(years.unique())
fig.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig.write_image('../figures/8_LSTM_preds_' + point_name + '.png')
fig.show()

### 5. Test CNN

Let's test a convolutional neural network (CNN) to compare to the LSTM.

In [16]:
# load the dataset, drop the series column
df = pd.read_csv(input_file)

df.drop(columns="series", inplace=True)
dataset=df['velocity'].values.reshape(-1, 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets, 70% test data, 30% training data
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = trainX[:-1]
trainY = trainY[:-1]

# reshape input to be [samples, time steps, features]
trainX_cnn = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX_cnn = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# create and fit the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(1))
cnn_model.compile(optimizer='adam', loss='mse')
print(cnn_model.summary())
cnn_model.fit(trainX_cnn, trainY, epochs=500, batch_size=32, validation_data=(testX_cnn, testY), verbose=0)

# make predictions
trainPredict_cnn = cnn_model.predict(trainX_cnn)
testPredict_cnn = cnn_model.predict(testX_cnn)

# invert predictions
trainPredict_cnn = scaler.inverse_transform(trainPredict_cnn)
testPredict_cnn = scaler.inverse_transform(testPredict_cnn)

# calculate root mean squared error
trainScore_cnn = math.sqrt(mean_squared_error(trainY, trainPredict_cnn[:,0]))
print('Train Score: %.2f RMSE' % (trainScore_cnn))
testScore_cnn = math.sqrt(mean_squared_error(testY, testPredict_cnn[:,0]))
print('Test Score: %.2f RMSE' % (testScore_cnn))
# shift train predictions for plotting
trainPredictPlot_cnn = np.empty_like(dataset)
trainPredictPlot_cnn[:, :] = np.nan
trainPredictPlot_cnn[look_back:len(trainPredict_cnn) + look_back, :] = trainPredict_cnn

# shift test predictions for plotting
testPredictPlot_cnn = np.empty_like(dataset)
testPredictPlot_cnn[:, :] = np.nan
testPredictPlot_cnn[len(trainPredict_cnn) + (look_back * 2) + 1:len(dataset) - 2, :] = testPredict_cnn

# Create traces
fig_cnn = go.Figure()
fig_cnn.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig_cnn.add_trace(go.Scatter(y=trainPredictPlot_cnn.flatten(), mode='lines', name='Train Predictions (CNN)'))
fig_cnn.add_trace(go.Scatter(y=testPredictPlot_cnn.flatten(), mode='lines', name='Test Predictions (CNN)'))

# Update layout for dark theme
fig_cnn.update_layout(title='CNN Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
years = pd.to_datetime(df['time']).dt.year

unique_years = sorted(years.unique())

fig_cnn.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig_cnn.write_image('../figures/8_CNN_preds_' + point_name + '.png')
fig_cnn.show()

248 248
104 104


/Users/cjense/miniconda3/envs/mlgeo-jensencc/lib/python3.10/site-packages/keras/layers/convolutional/base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 4, 64)          │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         6,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,693 (26.14 KB)

 Trainable params: 6,693 (26.14 KB)

 Non-trainable params: 0 (0.00 B)

None
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step
Train Score: 814.95 RMSE
Test Score: 837.87 RMSE


In [17]:
# Print RMSE scores for both models
print('LSTM Train Score: %.2f RMSE' % (trainScore_lstm))
print('LSTM Test Score: %.2f RMSE' % (testScore_lstm))
print('CNN Train Score: %.2f RMSE' % (trainScore_cnn))
print('CNN Test Score: %.2f RMSE' % (testScore_cnn))

# Plot predictions of both models
fig_comparison = go.Figure()
fig_comparison.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig_comparison.add_trace(go.Scatter(y=trainPredictPlot_lstm.flatten(), mode='lines', name='Train Predictions (LSTM)'))
fig_comparison.add_trace(go.Scatter(y=testPredictPlot_lstm.flatten(), mode='lines', name='Test Predictions (LSTM)'))
fig_comparison.add_trace(go.Scatter(y=trainPredictPlot_cnn.flatten(), mode='lines', name='Train Predictions (CNN)'))
fig_comparison.add_trace(go.Scatter(y=testPredictPlot_cnn.flatten(), mode='lines', name='Test Predictions (CNN)'))

# Update layout for dark theme
fig_comparison.update_layout(title='LSTM vs CNN Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
fig_comparison.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig_comparison.show()

LSTM Train Score: 37.46 RMSE
LSTM Test Score: 42.18 RMSE
CNN Train Score: 814.95 RMSE
CNN Test Score: 837.87 RMSE


### 6. Rolling Average Seasonality

The predictions are difficult to visually interpret. Let's see how well the two models capture seasonal changes in a 12 month rolling average plot.

In [18]:
# Calculate 12-month rolling average for actual data
df['velocity_rolling'] = df['velocity'].rolling(window=12).mean()

# Calculate 12-month rolling average for LSTM predictions
trainPredictPlot_lstm_rolling = pd.Series(trainPredictPlot_lstm.flatten()).rolling(window=12).mean().values
testPredictPlot_lstm_rolling = pd.Series(testPredictPlot_lstm.flatten()).rolling(window=12).mean().values

# Calculate 12-month rolling average for CNN predictions
trainPredictPlot_cnn_rolling = pd.Series(trainPredictPlot_cnn.flatten()).rolling(window=12).mean().values
testPredictPlot_cnn_rolling = pd.Series(testPredictPlot_cnn.flatten()).rolling(window=12).mean().values

# Create traces for rolling averages
fig_rolling = go.Figure()
fig_rolling.add_trace(go.Scatter(y=df['velocity_rolling'], mode='lines', name='Actual Data (12-month rolling)'))
fig_rolling.add_trace(go.Scatter(y=trainPredictPlot_lstm_rolling, mode='lines', name='Train Predictions (LSTM, 12-month rolling)'))
fig_rolling.add_trace(go.Scatter(y=testPredictPlot_lstm_rolling, mode='lines', name='Test Predictions (LSTM, 12-month rolling)'))
fig_rolling.add_trace(go.Scatter(y=trainPredictPlot_cnn_rolling, mode='lines', name='Train Predictions (CNN, 12-month rolling)'))
fig_rolling.add_trace(go.Scatter(y=testPredictPlot_cnn_rolling, mode='lines', name='Test Predictions (CNN, 12-month rolling)'))

# Update layout for dark theme
fig_rolling.update_layout(title='12-Month Rolling Average of Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
fig_rolling.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig_rolling.show()